ETL PROJECT BY MELINDA EUDY, KELLY ROSE, DARRIUS SUMMERS MARCH 2021

In [38]:
#dependencies
import pandas as pd
import requests
import json
from pprint import pprint
from config import api_key
from sqlalchemy import create_engine

api_key= api_key
headers = {'Authorization': 'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search'

STORE CSV INTO DATAFRAME ---KAGGLE - BREWERIES AND BEER PUBS IN THE US

In [39]:
csv_file = "Resources/7160_1.csv"
breweries_data_df = pd.read_csv(csv_file)
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,3205542939,56385,MN,http://www.10miletavern.com


In [40]:
# Remove locations with missing data
breweries_data_df.dropna(inplace=True)
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
10,3090 Shirley Dr,"Bars & Clubs, Breweries, Nightlife, Food & Drink",Jackson,US,us/mi/jackson/3090shirleydr/289346730,42.286010,-84.431260,127 Brewing,5172581346,49201,MI,https://www.facebook.com/127brewing
11,41 Lower Newton St,Brewers,Stalbans,US,us/vt/stalbans/41lowernewtonst,44.810713,-73.083558,14th Star Brewing Co Llc,8023931459,5478,VT,http://api.citygridmedia.com/content/places/v2...
15,189 Main St,"Restaurant, Restaurants, Bar, Brewery, and Res...",E Aurora,US,us/ny/eaurora/189mainst,42.767591,-78.627819,189 Public House,"7196528189, 7166528189",14052,NY,http://www.oneeightynine.com


In [41]:
# Delete Columns not required for final production database
del breweries_data_df["key"]
del breweries_data_df["long"]
del breweries_data_df["lat"]
del breweries_data_df["categories"]

# Reorder the columns to create better logic
updated_breweries_data_df = breweries_data_df[["name", "phones", "address", "city", "province", "postalCode", "websites"]]
updated_breweries_data_df.head()

,name,phones,address,city,province,postalCode,websites
2,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",830 W Bannock St,Boise,ID,83702,10barrel.com
3,10 Gallon Tours,"18174031832, 817 403-1832",1022 Texan Trl,Grapevine,TX,76051,http://www.10gallontours.com/
10,127 Brewing,5172581346,3090 Shirley Dr,Jackson,MI,49201,https://www.facebook.com/127brewing
11,14th Star Brewing Co Llc,8023931459,41 Lower Newton St,Stalbans,VT,5478,http://api.citygridmedia.com/content/places/v2...
15,189 Public House,"7196528189, 7166528189",189 Main St,E Aurora,NY,14052,http://www.oneeightynine.com


In [43]:
# Fix column names to prepare for merge with Yelp data
updated_breweries_data_df=updated_breweries_data_df.rename(columns ={"province": "state", "postalCode": "zip_code", "address": "address1"})
updated_breweries_data_df.set_index("address1", inplace = True)
updated_breweries_data_df.head()

,name,phones,city,state,zip_code,websites
address1,,,,,,
830 W Bannock St,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",Boise,ID,83702,10barrel.com
1022 Texan Trl,10 Gallon Tours,"18174031832, 817 403-1832",Grapevine,TX,76051,http://www.10gallontours.com/
3090 Shirley Dr,127 Brewing,5172581346,Jackson,MI,49201,https://www.facebook.com/127brewing
41 Lower Newton St,14th Star Brewing Co Llc,8023931459,Stalbans,VT,5478,http://api.citygridmedia.com/content/places/v2...
189 Main St,189 Public House,"7196528189, 7166528189",E Aurora,NY,14052,http://www.oneeightynine.com


In [54]:
# Create dictornary search from YELP for bars in North Carolina
params = {'term':'bars','location':'NC'}

# Making a get request to the API
response=requests.get(url, params=params, headers=headers).json()


In [86]:
# Print response from YELP API for Bars in North Carolina
bars_nc = json.dumps(response, indent = 4, sort_keys=True)

response['businesses']

[{'id': 'VlTR8dhK2Cz6N5kHB6DKRw',
  'alias': 'the-fairwood-226-charlotte-2',
  'name': 'The Fairwood 226',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/l7AtTYVpUJrmOfE2LdhMEQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/the-fairwood-226-charlotte-2?adjust_creative=1_exTwTpvzeU_FUI7UJu-g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1_exTwTpvzeU_FUI7UJu-g',
  'review_count': 34,
  'categories': [{'alias': 'beerbar', 'title': 'Beer Bar'},
   {'alias': 'sportsbars', 'title': 'Sports Bars'},
   {'alias': 'chicken_wings', 'title': 'Chicken Wings'}],
  'rating': 4.0,
  'coordinates': {'latitude': 35.20268, 'longitude': -80.87114},
  'transactions': [],
  'location': {'address1': '226 Fairwood Ave',
   'address2': None,
   'address3': '',
   'city': 'Charlotte',
   'zip_code': '28203',
   'country': 'US',
   'state': 'NC',
   'display_address': ['226 Fairwood Ave', 'Charlotte, NC 28203']},
  'phone': '+19803090463',
  'display_phone': '(9

In [68]:
for obj in response:
    print(obj)

businesses
total
region


In [119]:
id = []
address1 = []
price = []
rating = []
review_count = []

for row in response['businesses']:
    try:
        id.append(row['id']),
        address1.append(row['location']['address1']),
        price.append(row['price']),
        rating.append(row['rating']),
        review_count.append(row['review_count'])
        print(f"{address1}, {price}, {rating}, {review_count}")
    except:
        pass

['226 Fairwood Ave', '1225 Central Ave'], ['$'], [4.5], [130]
['226 Fairwood Ave', '1225 Central Ave', '1318 Pecan Ave', '1300 S Tryon St'], ['$', '$$'], [4.5, 4.0], [130, 122]
['226 Fairwood Ave', '1225 Central Ave', '1318 Pecan Ave', '1300 S Tryon St', '220 E Trade St'], ['$', '$$', '$$'], [4.5, 4.0, 4.0], [130, 122, 99]
['226 Fairwood Ave', '1225 Central Ave', '1318 Pecan Ave', '1300 S Tryon St', '220 E Trade St', '132 E Trade St', '1812 S Blvd', '144 Brevard Ct'], ['$', '$$', '$$', '$$'], [4.5, 4.0, 4.0, 4.5], [130, 122, 99, 68]
['226 Fairwood Ave', '1225 Central Ave', '1318 Pecan Ave', '1300 S Tryon St', '220 E Trade St', '132 E Trade St', '1812 S Blvd', '144 Brevard Ct', '14 Union St S', '2120 S Blvd', '4621 Wilkinson Blvd'], ['$', '$$', '$$', '$$', '$'], [4.5, 4.0, 4.0, 4.5, 3.5], [130, 122, 99, 68, 51]
['226 Fairwood Ave', '1225 Central Ave', '1318 Pecan Ave', '1300 S Tryon St', '220 E Trade St', '132 E Trade St', '1812 S Blvd', '144 Brevard Ct', '14 Union St S', '2120 S Blvd',

In [114]:
# create dataframe for YELP bars in North Carolina

bars_nc_df = pd.DataFrame(row).T
bars_nc_df.head()

ValueError: arrays must all be same length

In [22]:
# Create dictornary search from YELP for bars in South Carolina
params = {'term':'bars','location':'SC'}

# Making a get request to the API
response=requests.get(url, params=params, headers=headers).json()


In [23]:
# Print response from YELP API for Bars in South Carolina
bars_nc = json.dumps(response, indent = 4, sort_keys=True)

response['businesses']

('{\n'
 '    "businesses": [\n'
 '        {\n'
 '            "alias": "the-whig-columbia",\n'
 '            "categories": [\n'
 '                {\n'
 '                    "alias": "divebars",\n'
 '                    "title": "Dive Bars"\n'
 '                }\n'
 '            ],\n'
 '            "coordinates": {\n'
 '                "latitude": 34.0015114,\n'
 '                "longitude": -81.033505\n'
 '            },\n'
 '            "display_phone": "(803) 931-8852",\n'
 '            "distance": 4144.845632578871,\n'
 '            "id": "eL-gvkAzpoGHMmgKURSJDw",\n'
 '            "image_url": '
 '"https://s3-media3.fl.yelpcdn.com/bphoto/o2ir-8jXxMlbG8QhoB3OSw/o.jpg",\n'
 '            "is_closed": false,\n'
 '            "location": {\n'
 '                "address1": "1200 Main St",\n'
 '                "address2": "",\n'
 '                "address3": "",\n'
 '                "city": "Columbia",\n'
 '                "country": "US",\n'
 '                "display_address": [\n'
 '  

In [ ]:
address1 = []
price = []
rating = []
review_count = []

for row in response['businesses']:
    try:
        address1.append(row['location']['address1']),
        price.append(row['price']),
        rating.append(row['rating']),
        review_count.append(row['review_count'])
        print(f"{address1}, {price}, {rating}, {review_count}")
    except:
        pass